# Conditional Entropy

### References

- [DataScience - StackExchange: Conditional entropy calculation in python, H(Y|X)](https://datascience.stackexchange.com/questions/58565/conditional-entropy-calculation-in-python-hyx)

In [7]:
import pandas as pd
import numpy as np

### data

In [2]:
attrNms = ["x1", "x2", "x3", "x4", "x5"]
data = pd.DataFrame(
    [
        [1, 0, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1],
    ],
    columns=attrNms,
)
data["y"] = [1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

### estimation

In [8]:
##Entropy
def entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en


#Joint Entropy
def jEntropy(Y,X):
    """
    H(Y;X)
    Reference: https://en.wikipedia.org/wiki/Joint_entropy
    """
    YX = np.c_[Y,X]
    return entropy(YX)

#Conditional Entropy
def cEntropy(Y, X):
    """
    conditional entropy = Joint Entropy - Entropy of X
    H(Y|X) = H(Y;X) - H(X)
    Reference: https://en.wikipedia.org/wiki/Conditional_entropy
    """
    return jEntropy(Y, X) - entropy(X)


#Information Gain
def gain(Y, X):
    """
    Information Gain, I(Y;X) = H(Y) - H(Y|X)
    Reference: https://en.wikipedia.org/wiki/Information_gain_in_decision_trees#Formal_definition
    """
    return entropy(Y) - cEntropy(Y,X)

In [16]:
y = data["y"].values
X = data[attrNms].values

In [17]:
entropy(y), cEntropy(y,X), gain(y, X)

(0.9709505944546686, 0.20000000000000018, 0.7709505944546684)